# Share components, environments and models across workspaces

This is the companion notebook for the article on sharing components, environments and models across workspaces: https://learn.microsoft.com/en-us/azure/machine-learning/how-to-share-models-pipelines-across-workspaces-with-registries 

### Prerequisites
Review the prerequisites section in the article: https://learn.microsoft.com/en-us/azure/machine-learning/how-to-share-models-pipelines-across-workspaces-with-registries?tabs=python#prerequisites. To summarize, in addition to the Python SDK, you need an AzureML registry and an AzureML workspace in a region that is supported by the workspace.


### Scenarios

There are two scenarios where you'd want to use the same set of models, components and environments in multiple workspaces:

* Cross-workspace MLOps: You're training a model in a dev workspace and need to deploy it to test and prod workspaces. In this case you, want to have end-to-end lineage between endpoints to which the model is deployed in test or prod workspaces and the training job, metrics, code, data and environment that was used to train the model in the dev workspace.
* Share and reuse models and pipelines across different teams: Sharing and reuse improve collaboration and productivity. In this scenario, you may want to publish a trained model and the associated components and environments used to train it to a central catalog. From there, colleagues from other teams can search and reuse the assets you shared in their own experiments.

### Goals
* Create an environment and component in the registry.
* Use the component from registry to submit a model training job in a workspace.
* Register the trained model in the registry.
* Deploy the model from the registry to an online-endpoint in the workspace, then submit an inference request.


In [1]:
# Import required libraries
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential

from azure.ai.ml import MLClient, Input, Output
from azure.ai.ml.dsl import pipeline
from azure.ai.ml import load_component
from azure.ai.ml.entities import (
    Environment,
    BuildContext,
    Model,
    ManagedOnlineEndpoint,
    ManagedOnlineDeployment,
    CodeConfiguration,
)
from azure.ai.ml.constants import AssetTypes
import time, datetime, os

# print the sdk version - you many want to share this in the issue you will report if parts of this notebook don't work
!pip show azure-ai-ml

Class SystemCreatedStorageAccount: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class SystemCreatedAcrAccount: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.


Name: azure-ai-ml
Version: 1.0.0
Summary: Microsoft Azure Machine Learning Client Library for Python
Home-page: https://github.com/Azure/azure-sdk-for-python
Author: Microsoft Corporation
Author-email: azuresdkengsysadmins@microsoft.com
License: MIT License
Location: /root/miniconda3/lib/python3.8/site-packages
Requires: typing-extensions, azure-common, strictyaml, pydash, pyyaml, azure-storage-file-share, isodate, azure-core, marshmallow, tqdm, colorama, msrest, azure-mgmt-core, jsonschema, pyjwt, azure-storage-blob, azure-storage-file-datalake
Required-by: 


### Setup authentication

We are using `DefaultAzureCredential` to get access to workspace. When an access token is needed, it requests one using multiple identities(`EnvironmentCredential, ManagedIdentityCredential, SharedTokenCacheCredential, VisualStudioCodeCredential, AzureCliCredential, AzurePowerShellCredential`) in turn, stopping when one provides a token.
Reference [here](https://docs.microsoft.com/en-us/python/api/azure-identity/azure.identity.defaultazurecredential?view=azure-python) for more information.

`DefaultAzureCredential` should be capable of handling most Azure SDK authentication scenarios. 
Reference [here](https://docs.microsoft.com/en-us/python/api/azure-identity/azure.identity?view=azure-python) for all available credentials if it does not work for you.  

In [2]:
try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    credential = InteractiveBrowserCredential()

## Connect to a workspace and registry

Most samples create one client to connect to the workspace. However, in this sample, you need two clients. First client, called `ml_client_workspace_dev`, will be used to connect to a workspace and run jobs or deploy endpoints. Second client, called `ml_client_registry` will be used to connect to the registry to create components, environments and models.

Replace the following:
* `<SUBSCRIPTION_ID>`
* `<RESOURCE_GROUP>`
* `<AML_WORKSPACE_NAME>`
* `<REGISTRY_NAME>`
 

In [3]:
ml_client_workspace_dev = MLClient(
    credential=credential,
    subscription_id="59032f82-256a-462d-a725-92fcf3858502",
    resource_group_name="registries",
    workspace_name="ContosoMLDev",
)
print(ml_client_workspace_dev)

ml_client_workspace_prod = MLClient(
    credential=credential,
    subscription_id="59032f82-256a-462d-a725-92fcf3858502",
    resource_group_name="registries",
    workspace_name="ContosoMLProd",
)
print(ml_client_workspace_prod)

ml_client_registry = MLClient(credential=credential, registry_name="ContosoMLRegistry")
print(ml_client_registry)

Class RegistryOperations: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.


MLClient(credential=<azure.identity._credentials.default.DefaultAzureCredential object at 0x7f7aa01e57c0>,
         subscription_id=59032f82-256a-462d-a725-92fcf3858502,
         resource_group_name=registries,
         workspace_name=ContosoMLDev)
MLClient(credential=<azure.identity._credentials.default.DefaultAzureCredential object at 0x7f7aa01e57c0>,
         subscription_id=59032f82-256a-462d-a725-92fcf3858502,
         resource_group_name=registries,
         workspace_name=ContosoMLProd)
MLClient(credential=<azure.identity._credentials.default.DefaultAzureCredential object at 0x7f7aa01e57c0>,
         subscription_id=59032f82-256a-462d-a725-92fcf3858502,
         resource_group_name=registries,
         workspace_name=None)


### Create a version number and setup root directory 
Make sure that you set the version number to something unique if this notebook has been run before. You can use the timestamp to generate a unique version number, the sample code for which is commented out. This will prevent any name and version conflicts when creating assets.

Set the root directory in which the YAML definitions of the components, environments, etc. are present.

In [4]:
import time
import sys

# version = str(123456)
version = str(int(time.time()))
print("version: ", version)

parent_dir = os.path.abspath(
    os.path.join(
        sys.path[0],
        "../../../../cli/jobs/pipelines-with-components/nyc_taxi_data_regression",
    )
)

version:  1667421519


### Create environment in registry

You will use a docker file to create the environment. The docker file has base python image and few python dependencies required to run Scikit Learn training jobs. This notebook: [../environment/environment.ipynb](../environment/environment.ipynb)has more samples for environment create.

Note that we use the `ml_client_registry` client because we plan to create the environment in registry. The syntax for creating environment in a workspace or registry are identical. You just use a client that is specific to the target - workspace or registry.

In [5]:
env_docker_context = Environment(
    build=BuildContext(path=os.path.join(parent_dir, "env_train")),
    name="SKLearnEnv",
    version=version,
    description="Scikit Learn environment",
)
ml_client_registry.environments.create_or_update(env_docker_context)

Uploading env_train (0.0 MBs): 100%|██████████| 491/491 [00:00<00:00, 8210.85it/s]




Environment({'is_anonymous': False, 'auto_increment_version': False, 'name': 'SKLearnEnv', 'description': 'Scikit Learn environment', 'tags': {}, 'properties': {}, 'id': 'azureml://registries/ContosoMLRegistry/environments/SKLearnEnv/versions/1667421519', 'Resource__source_path': None, 'base_path': '/mnt/c/CODE/REPOS/azureml-examples/sdk/python/assets/assets-in-registry', 'creation_context': <azure.ai.ml.entities._system_data.SystemData object at 0x7f7a8982d1c0>, 'serialize': <msrest.serialization.Serializer object at 0x7f7a89785670>, 'version': '1667421519', 'latest_version': None, 'conda_file': None, 'image': 'conto91e1ad98361c467b8de.azurecr.io/sklearnenv_1667421519_97869441-2af7-5f8c-8f3c-bd15c3fa2dce', 'build': None, 'inference_config': None, 'os_type': 'Linux', 'arm_type': 'environment_version', 'conda_file_path': None, 'path': None, 'datastore': None, 'upload_hash': None, 'translated_conda_file': None})

### Get environment from registry

Get the environment using the `ml_client_registry` client. The syntax for getting environment in a workspace or registry are identical. You just use a client that is specific to the target - workspace or registry.

You will use this environment in the next step to create a component in the registry.

In [6]:
env_from_registry = ml_client_registry.environments.get(
    name="SKLearnEnv", version=version
)
print(env_from_registry)

Environment({'is_anonymous': False, 'auto_increment_version': False, 'name': 'SKLearnEnv', 'description': 'Scikit Learn environment', 'tags': {}, 'properties': {}, 'id': 'azureml://registries/ContosoMLRegistry/environments/SKLearnEnv/versions/1667421519', 'Resource__source_path': None, 'base_path': '/mnt/c/CODE/REPOS/azureml-examples/sdk/python/assets/assets-in-registry', 'creation_context': <azure.ai.ml.entities._system_data.SystemData object at 0x7f7a8981db50>, 'serialize': <msrest.serialization.Serializer object at 0x7f7aa0cfeb50>, 'version': '1667421519', 'latest_version': None, 'conda_file': None, 'image': 'conto91e1ad98361c467b8de.azurecr.io/sklearnenv_1667421519_97869441-2af7-5f8c-8f3c-bd15c3fa2dce', 'build': None, 'inference_config': None, 'os_type': 'Linux', 'arm_type': 'environment_version', 'conda_file_path': None, 'path': None, 'datastore': None, 'upload_hash': None, 'translated_conda_file': None})


### Create component in registry

You will use the [`train.yml`](../../../cli/jobs/pipelines-with-components/nyc_taxi_data_regression/train.yml) component YAML defined in `cli/jobs/pipelines-with-components/nyc_taxi_data_regression` for this. This component runs a Scikit Learn training python script. The `train.yml` refers to the AzureML curated environment for the Scikit Learn framework: `AzureML-sklearn-0.24-ubuntu18`, but you will over ride this to use the Scikit Learn environment you created in the previous step.

A similar sample notebook shows how to create these components in workspaces instead of registry, in which case you can use those components only in the specific workspace: https://github.com/Azure/azureml-examples/blob/main/sdk/jobs/pipelines/1e_pipeline_with_registered_components/pipeline_with_registered_components.ipynb


Use the `ml_client_registry` client to create the component in the registry. The syntax for creating component in a workspace or registry are identical. You just use a client that is specific to the target - workspace or registry.

In [7]:
# load component definition from YAML
print(parent_dir)
train_model = load_component(source=os.path.join(parent_dir, "train.yml"))
# print the component as yaml
print(train_model)

# change environment reference to the environment created in registry
train_model.environment = env_from_registry

# changing the version number is optional, but useful if a component with same name and version already exist in registry
train_model.version = version

print(train_model)
ml_client_registry.components.create_or_update(train_model)

/mnt/c/CODE/REPOS/azureml-examples/cli/jobs/pipelines-with-components/nyc_taxi_data_regression
$schema: https://azuremlschemas.azureedge.net/latest/commandComponent.schema.json
name: train_linear_regression_model
version: '1'
display_name: TrainLinearRegressionModel
type: command
inputs:
  training_data:
    type: uri_folder
  test_split_ratio:
    type: number
    default: '0.2'
    max: '1.0'
    min: '0.0'
outputs:
  model_output:
    type: mlflow_model
  test_data:
    type: uri_folder
command: python train.py  --training_data ${{inputs.training_data}}  --test_data ${{outputs.test_data}}  --model_output
  ${{outputs.model_output}} --test_split_ratio ${{inputs.test_split_ratio}}
environment: azureml:AzureML-sklearn-0.24-ubuntu18.04-py37-cpu@latest
code: ./train_src
is_deterministic: true
tags: {}

$schema: https://azuremlschemas.azureedge.net/latest/commandComponent.schema.json
name: train_linear_regression_model
version: '1667421519'
display_name: TrainLinearRegressionModel
type: c

Uploading train_src (0.0 MBs): 100%|██████████| 2521/2521 [00:00<00:00, 50075.49it/s]


Creating/updating registry component train_linear_regression_model with version 1667421519 

.

Done (0m 6s)


CommandComponent({'auto_increment_version': False, 'source': 'REMOTE.REGISTRY', 'is_anonymous': False, 'name': 'train_linear_regression_model', 'description': None, 'tags': {}, 'properties': {}, 'id': 'azureml://registries/ContosoMLRegistry/components/train_linear_regression_model/versions/1667421519', 'Resource__source_path': None, 'base_path': './', 'creation_context': <azure.ai.ml._restclient.v2021_10_01_dataplanepreview.models._models_py3.SystemData object at 0x7f7a886c6af0>, 'serialize': <msrest.serialization.Serializer object at 0x7f7a886c69a0>, 'command': 'python train.py  --training_data ${{inputs.training_data}}  --test_data ${{outputs.test_data}}  --model_output ${{outputs.model_output}} --test_split_ratio ${{inputs.test_split_ratio}}', 'code': 'azureml://registries/ContosoMLRegistry/codes/80b032fd-1989-4aef-9010-d18d614bcde0/versions/1', 'environment_variables': None, 'environment': 'azureml://registries/ContosoMLRegistry/environments/SKLearnEnv/versions/1667421519', 'distri

### Get component from Registry

Get the component using the `ml_client_registry` client. The syntax for getting component from a workspace or registry are identical. You just use a client that is specific to the target - workspace or registry.

You will use this component in the next step to run a pipeline job to train a model.

In [8]:
train_component_from_registry = ml_client_registry.components.get(
    name="train_linear_regression_model", version=version
)
print(train_component_from_registry)

$schema: https://azuremlschemas.azureedge.net/latest/commandComponent.schema.json
name: train_linear_regression_model
version: '1667421519'
display_name: TrainLinearRegressionModel
type: command
inputs:
  training_data:
    type: uri_folder
  test_split_ratio:
    type: number
    default: '0.2'
    max: '1.0'
    min: '0.0'
outputs:
  model_output:
    type: mlflow_model
  test_data:
    type: uri_folder
command: python train.py  --training_data ${{inputs.training_data}}  --test_data ${{outputs.test_data}}  --model_output
  ${{outputs.model_output}} --test_split_ratio ${{inputs.test_split_ratio}}
environment: azureml://registries/ContosoMLRegistry/environments/SKLearnEnv/versions/1667421519
code: azureml://registries/ContosoMLRegistry/codes/80b032fd-1989-4aef-9010-d18d614bcde0/versions/1
resources:
  instance_count: 1
creation_context:
  created_at: '2022-11-02T20:42:51.763294+00:00'
  created_by: Manoj Bableshwar
  created_by_type: User
  last_modified_at: '2022-11-02T20:42:51.763294

### Create a pipeline job using component from registry

Review this page to learn how to use pipelines and components: https://github.com/Azure/azureml-examples/tree/main/sdk/python/jobs/pipelines. 

You will create a pipeline job that uses the training component created in the previous step using the Python DSL for pipelines. 

Make sure your workspace has a compute with the name `cpu-cluster` or update the compute name here: `pipeline_job.settings.default_compute = `

In [9]:
@pipeline()
def pipeline_with_registered_components(training_data):
    train_job = train_component_from_registry(
        training_data=training_data,
    )


pipeline_job = pipeline_with_registered_components(
    training_data=Input(type="uri_folder", path=parent_dir + "/data_transformed/"),
)
pipeline_job.settings.default_compute = "cpu-cluster"
print(pipeline_job)

display_name: pipeline_with_registered_components
type: pipeline
inputs:
  training_data:
    type: uri_folder
    path: azureml:/mnt/c/CODE/REPOS/azureml-examples/cli/jobs/pipelines-with-components/nyc_taxi_data_regression/data_transformed/
outputs: {}
jobs:
  train_job:
    type: command
    inputs:
      training_data:
        path: ${{parent.inputs.training_data}}
    outputs: {}
    code: azureml://registries/ContosoMLRegistry/codes/80b032fd-1989-4aef-9010-d18d614bcde0/versions/1
    resources:
      instance_count: 1
      properties: {}
    component:
      $schema: https://azuremlschemas.azureedge.net/latest/commandComponent.schema.json
      name: train_linear_regression_model
      version: '1667421519'
      display_name: TrainLinearRegressionModel
      type: command
      inputs:
        training_data:
          type: uri_folder
        test_split_ratio:
          type: number
          default: '0.2'
          max: '1.0'
          min: '0.0'
      outputs:
        model_o

### Run pipeline job using a component from registry

Submit pipeline job and wait for it to complete. Notice that you are using the workspace client: `ml_client_workspace_dev` to run the pipeline job. This job is running a component that is not available in your workspace but is coming from a registry. This way, you can run this job in any workspace you have access to. This is useful when you want to develop a pipeline in th `dev` workspace with some sample data and run the pipeline in the `prod` workspace with actual data. This is also helpful if you want to share the components you develop with other teams in your organization who may be using a different workspace. 
To summarize, you can submit this job to different workspaces such as `dev`, `test` or `prod` by creating different ML clients for each of those workspaces.

In [10]:
pipeline_job = ml_client_workspace_dev.jobs.create_or_update(
    pipeline_job,
    experiment_name="sdk_job_component_from_registry",
    skip_validation=True,
)
ml_client_workspace_dev.jobs.stream(pipeline_job.name)
pipeline_job = ml_client_workspace_dev.jobs.get(pipeline_job.name)
pipeline_job

Uploading data_transformed (1.26 MBs): 100%|██████████| 1257088/1257088 [00:00<00:00, 3600190.93it/s]




RunId: mango_fig_jggff1cp33
Web View: https://ml.azure.com/runs/mango_fig_jggff1cp33?wsid=/subscriptions/59032f82-256a-462d-a725-92fcf3858502/resourcegroups/registries/workspaces/ContosoMLDev

Streaming logs/azureml/executionlogs.txt

[2022-11-02 20:43:08Z] Submitting 1 runs, first five are: 5961d2d3:801ecf9a-e52a-4129-b19f-3abed69a2383
[2022-11-02 20:45:06Z] Completing processing run id 801ecf9a-e52a-4129-b19f-3abed69a2383.

Execution Summary
RunId: mango_fig_jggff1cp33
Web View: https://ml.azure.com/runs/mango_fig_jggff1cp33?wsid=/subscriptions/59032f82-256a-462d-a725-92fcf3858502/resourcegroups/registries/workspaces/ContosoMLDev



Experiment,Name,Type,Status,Details Page
sdk_job_component_from_registry,mango_fig_jggff1cp33,pipeline,Completed,Link to Azure Machine Learning studio


### Create model in registry

You will now obtain the model trained by the pipeline job in the above step and create the model in the registry. For completeness, we are showing two options here.
* First option shows to create a model in registry from job output without downloading it. This option is recommended when you want to track the lineage between the training job and the model. You will create a model directly from the job output (without downloading it) in your workspace and then copy the model from workspace to registry. 
* Second option shows how to create a model in registry from local files. In this case you will download the model from the job output. This option is helpful if you have an existing model from some external source and want to host it in the registry to be shared with many workspaces.

Review this notebook to learn the different model types and how to create them in a workspace: [../../assets/model/model.ipynb](../model/model.ipynb). In the below example, you will work with a `mlflow_model` that will help you deploy this model for inference without writing any scoring scripts.

### Create model in workspace and copy it to registry

Step a: Get the model path from job output. Note that you use the `ml_client_workspace_dev` to get the model path from job output in a workspace.

In [11]:
jobs = ml_client_workspace_dev.jobs.list(parent_job_name=pipeline_job.name)
for job in jobs:
    if job.display_name == "train_job":
        print(job.name)
        model_path_from_job = (
            "azureml://jobs/{job_name}/outputs/artifacts/paths/model".format(
                job_name=job.name
            )
        )

print(model_path_from_job)

801ecf9a-e52a-4129-b19f-3abed69a2383
azureml://jobs/801ecf9a-e52a-4129-b19f-3abed69a2383/outputs/artifacts/paths/model


### Create model in workspace and copy it to registry
Step b: Create model in workspace from job output. Note that you use the `ml_client_workspace_dev` to create the model in workspace.

In [12]:
mlflow_model = Model(
    path=model_path_from_job,
    type=AssetTypes.MLFLOW_MODEL,
    name="nyc-taxi-model",
    version=version,
    description="MLflow model created from job output",
)
print(mlflow_model)
ml_client_workspace_dev.models.create_or_update(mlflow_model)

Model({'job_name': None, 'is_anonymous': False, 'auto_increment_version': False, 'name': 'nyc-taxi-model', 'description': 'MLflow model created from job output', 'tags': {}, 'properties': {}, 'id': None, 'Resource__source_path': None, 'base_path': '/mnt/c/CODE/REPOS/azureml-examples/sdk/python/assets/assets-in-registry', 'creation_context': None, 'serialize': <msrest.serialization.Serializer object at 0x7f7a885d1340>, 'version': '1667421519', 'latest_version': None, 'path': 'azureml://jobs/801ecf9a-e52a-4129-b19f-3abed69a2383/outputs/artifacts/paths/model', 'datastore': None, 'utc_time_created': None, 'flavors': None, 'arm_type': 'model_version', 'type': 'mlflow_model'})


Model({'job_name': '801ecf9a-e52a-4129-b19f-3abed69a2383', 'is_anonymous': False, 'auto_increment_version': False, 'name': 'nyc-taxi-model', 'description': 'MLflow model created from job output', 'tags': {}, 'properties': {}, 'id': '/subscriptions/59032f82-256a-462d-a725-92fcf3858502/resourceGroups/registries/providers/Microsoft.MachineLearningServices/workspaces/ContosoMLDev/models/nyc-taxi-model/versions/1667421519', 'Resource__source_path': None, 'base_path': '/mnt/c/CODE/REPOS/azureml-examples/sdk/python/assets/assets-in-registry', 'creation_context': <azure.ai.ml.entities._system_data.SystemData object at 0x7f7a88697fd0>, 'serialize': <msrest.serialization.Serializer object at 0x7f7a88642130>, 'version': '1667421519', 'latest_version': None, 'path': 'azureml://subscriptions/59032f82-256a-462d-a725-92fcf3858502/resourceGroups/registries/workspaces/ContosoMLDev/datastores/workspaceartifactstore/paths/ExperimentRun/dcid.801ecf9a-e52a-4129-b19f-3abed69a2383/model', 'datastore': None, 

### Create model in workspace and copy it to registry

Step c: Get the model from workspace, prepare to copy it to registry and create the model registry using the `model_ready_to_copy` object. Note that you both the clients here: First, `ml_client_workspace_dev` client to get the model from workspace and prepare the `model_ready_to_copy` object. Second, `ml_client_registry` to create the model in registry using the `model_ready_to_copy` object.

In [13]:
# fetch the model from workspace
model_in_workspace = ml_client_workspace_dev.models.get(
    name="nyc-taxi-model", version=version
)
print("workspace model:\n\n", model_in_workspace)
# change the format such that the registry understands the model (when you print the model_ready_to_copy object, notice the asset id
model_ready_to_copy = ml_client_workspace_dev.models._prepare_to_copy(model_in_workspace)
print("\n\nmodel ready to copy:\n\n", model_ready_to_copy)
# copy the model from registry to workspace
ml_client_registry.models.create_or_update(model_ready_to_copy).wait()

workspace model:

 Model({'job_name': '801ecf9a-e52a-4129-b19f-3abed69a2383', 'is_anonymous': False, 'auto_increment_version': False, 'name': 'nyc-taxi-model', 'description': 'MLflow model created from job output', 'tags': {}, 'properties': {}, 'id': '/subscriptions/59032f82-256a-462d-a725-92fcf3858502/resourceGroups/registries/providers/Microsoft.MachineLearningServices/workspaces/ContosoMLDev/models/nyc-taxi-model/versions/1667421519', 'Resource__source_path': None, 'base_path': '/mnt/c/CODE/REPOS/azureml-examples/sdk/python/assets/assets-in-registry', 'creation_context': <azure.ai.ml.entities._system_data.SystemData object at 0x7f7a886cd790>, 'serialize': <msrest.serialization.Serializer object at 0x7f7a88697340>, 'version': '1667421519', 'latest_version': None, 'path': 'azureml://subscriptions/59032f82-256a-462d-a725-92fcf3858502/resourceGroups/registries/workspaces/ContosoMLDev/datastores/workspaceartifactstore/paths/ExperimentRun/dcid.801ecf9a-e52a-4129-b19f-3abed69a2383/model', 

### Deploy model from registry to online endpoint in workspace

You will deploy the model to an online endpoint and submit some sample inference requests in this section. Note that just like jobs, endpoints that host models are specific to a workspace. You can deploy the a model from a registry to many workspaces. This helps you develop a model in `dev` workspace, share it with a registry, and then deploy it to `test` and `prod` workspaces. 

### Get model from registry

Use the `ml_client_registry` client to get the model created in previous section from the registry. The syntax for creating component in a workspace or registry are identical. You just use a client that is specific to the target - workspace or registry.

In [14]:
mlflow_model_from_registry = ml_client_registry.models.get(
    name="nyc-taxi-model", version=version
)
print(mlflow_model_from_registry)

Model({'job_name': '801ecf9a-e52a-4129-b19f-3abed69a2383', 'is_anonymous': False, 'auto_increment_version': False, 'name': 'nyc-taxi-model', 'description': 'MLflow model created from job output', 'tags': {}, 'properties': {}, 'id': 'azureml://registries/ContosoMLRegistry/models/nyc-taxi-model/versions/1667421519', 'Resource__source_path': None, 'base_path': '/mnt/c/CODE/REPOS/azureml-examples/sdk/python/assets/assets-in-registry', 'creation_context': <azure.ai.ml.entities._system_data.SystemData object at 0x7f7a885bb3a0>, 'serialize': <msrest.serialization.Serializer object at 0x7f7a885bbbe0>, 'version': '1667421519', 'latest_version': None, 'path': 'https://contob50e9c08009542a9a4b.blob.core.windows.net/contosomlr-02c48c36-14be-54e8-8474-8d64686193d7/ExperimentRun/dcid.801ecf9a-e52a-4129-b19f-3abed69a2383/model', 'datastore': None, 'utc_time_created': None, 'flavors': {'python_function': {'env': 'conda.yaml', 'loader_module': 'mlflow.sklearn', 'model_path': 'model.pkl', 'predict_fn': 

### Create an online endpoint 

Create an online endpoint to deploy the model

In [20]:
online_endpoint_name = "endpoint-1" + version
# create an online endpoint
endpoint = ManagedOnlineEndpoint(
    name=online_endpoint_name,
    description="this is a sample online endpoint for mlflow model",
    auth_mode="key",
)
ml_client_workspace_prod.begin_create_or_update(endpoint).wait()

HttpResponseError: (None) InternalServerError: Internal error. Please see troubleshooting guide, available here: https://aka.ms/oe-tsg#error-internalservererror
Code: None
Message: InternalServerError: Internal error. Please see troubleshooting guide, available here: https://aka.ms/oe-tsg#error-internalservererror

### Deploy the model from registry to the online endpoint

In [ ]:
# create a demo deployment
demo_deployment = ManagedOnlineDeployment(
    name="demo",
    endpoint_name=online_endpoint_name,
    model=mlflow_model_from_registry.id,
    instance_type="Standard_F4s_v2",
    instance_count=1,
)
ml_client_workspace_prod.online_deployments.begin_create_or_update(demo_deployment).wait()

endpoint.traffic = {"demo": 100}
ml_client_workspace_prod.begin_create_or_update(endpoint)

Check: endpoint endpoint-1667421519 exists
data_collector is not a known attribute of class <class 'azure.ai.ml._restclient.v2022_02_01_preview.models._models_py3.ManagedOnlineDeployment'> and will be ignored


HttpResponseError: (BadRequest) The request is invalid.
Code: BadRequest
Message: The request is invalid.
Exception Details:	(InferencingClientCreateDeploymentFailed) InferencingClient HttpRequest error, error detail: {"errors":{"":["This endpoint has not been created successfully. Please recreate endpoint and then try again to create deployment."]},"type":"https://tools.ietf.org/html/rfc7231#section-6.5.1","title":"One or more validation errors occurred.","status":400,"traceId":"00-4748b3638adb04b8043851fb6f3dc500-fce74b9cf8e1fc11-00"}
	Code: InferencingClientCreateDeploymentFailed
	Message: InferencingClient HttpRequest error, error detail: {"errors":{"":["This endpoint has not been created successfully. Please recreate endpoint and then try again to create deployment."]},"type":"https://tools.ietf.org/html/rfc7231#section-6.5.1","title":"One or more validation errors occurred.","status":400,"traceId":"00-4748b3638adb04b8043851fb6f3dc500-fce74b9cf8e1fc11-00"}
Additional Information:T

### Test the deployment

This section needs a sample request file `scoring-data.json` which is available in the root directory initialized in the beginning of this notebook: [../../../cli/jobs/pipelines-with-components/nyc_taxi_data_regression/scoring-data.json](../../../cli/jobs/pipelines-with-components/nyc_taxi_data_regression/scoring-data.json)

In [ ]:
# test the  deployment with some sample data
ml_client_workspace_prod.online_endpoints.invoke(
    endpoint_name=online_endpoint_name,
    deployment_name="demo",
    request_file=parent_dir + "/scoring-data.json",
)

### Clean up resources

#### delete online endpoint 

In [ ]:
print(f"online_endpoint_name: {online_endpoint_name}")
#ml_client_workspace_prod.online_endpoints.begin_delete(name=online_endpoint_name).wait()